In [2]:
import pandas as pd
import datetime
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

%matplotlib inline

In [3]:
url = 'https://raw.githubusercontent.com/ngophuong/VNStock/main/excel_tcb.csv'
df = pd.read_csv(url, index_col = 0)
df['datetime']=pd.to_datetime(df['<DTYYYYMMDD>'],format='%Y%m%d')
df['Week_no']= df['datetime'].dt.week
df['Year_no']= df['datetime'].dt.year
df['Period'] = df['Year_no'].astype(str) + str("-") + df['Week_no'].astype(str)
df.head(3)

<ipython-input-3-7ef9f6d38ffb>:4: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



,<DTYYYYMMDD>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,<Open>,<High>,<Low>,<Close>,<VolumeDeal>,<VolumeFB>,<VolumeFS>,datetime,Week_no,Year_no,Period
<Ticker>,,,,,,,,,,,,,,,,,
TCB,20210409,41.7,42.00,41.5,41.60,8022500,41.7,42.00,41.5,41.60,100000,100000,100000,2021-04-09,14,2021,2021-14
TCB,20210408,42.4,42.45,41.7,41.95,7518800,42.4,42.45,41.7,41.95,0,0,0,2021-04-08,14,2021,2021-14
TCB,20210407,42.0,42.55,41.8,42.20,7728400,42.0,42.55,41.8,42.20,77000,31400,30000,2021-04-07,14,2021,2021-14


In [4]:
df =df[['<Close>','datetime','<Volume>','Period','Year_no']]
df = df.head(1500)

In [17]:
#plot linechart for stock price
short_rolling = df['<Close>'].rolling(window=20).mean()

ema_long = df['<Close>'].ewm(span=50, adjust=False).mean()
ema_short = df['<Close>'].ewm(span=20, adjust=False).mean()
line_price = df['<Close>']
fig = go.Figure()
fig.update_layout(  autosize=False,   width=1200,    height=600)
#px.line(df_hpg, x='datetime', y='<Close>', labels={'x':'Date', 'y':'Price'})

fig.add_trace(go.Scatter(x=df.datetime, y=line_price, 
                        mode='lines', name='TCB'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_short, 
                        mode='lines', name='Span 20-days EMA'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_long, 
                        mode='lines', name='Span 50-days EMA'))



In [23]:
#px.violin(df_tips, y="tip", x="smoker", color="sex", box=True, points="all",
#          hover_data=df_tips.columns)

fig = go.Figure()
px.violin(df,x='Year_no',y='<Close>',box =True, points='all',
         hover_data=df.columns, title = 'VND Violin chart', width=900, height=600)

In [7]:
df.tail(3)

,<Close>,datetime,<Volume>,Period,Year_no
<Ticker>,,,,,
TCB,92.0,2018-06-06,1605480,2018-23,2018
TCB,96.0,2018-06-05,1357500,2018-23,2018
TCB,102.4,2018-06-04,2811840,2018-23,2018


In [8]:
#HDFC_df[‘Day_Perc_Change’] = HDFC_df[‘Adj Close’].pct_change()*100
df['Daily_return']=df['<Close>'].pct_change()*100
df.dropna(axis = 0, inplace = True)
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return
<Ticker>,,,,,,
TCB,41.95,2021-04-08,7518800,2021-14,2021,0.841346
TCB,42.20,2021-04-07,7728400,2021-14,2021,0.595948
TCB,42.20,2021-04-06,14957000,2021-14,2021,0.000000


In [9]:
fig = go.Figure()
px.line(df, x='datetime', y='Daily_return', labels={'x':'Date', 'y':'Daily_Return_%'})

In [10]:
df['Daily_return'].describe()

count    715.000000
mean       0.262203
std        7.322424
min       -6.504065
25%       -0.961538
50%        0.000000
75%        0.844781
max      187.744227
Name: Daily_return, dtype: float64

In [11]:
fig = go.Figure()
px.histogram(df, x="Daily_return",histnorm='percent')
#ff.create_distplot(df, group_labels)

In [12]:
df['<Close>'].describe()

count    715.000000
mean      27.066084
std       13.748216
min       14.900000
25%       21.400000
50%       23.700000
75%       26.900000
max      109.000000
Name: <Close>, dtype: float64

In [13]:
dfig = go.Figure()
px.histogram(df.head(60), x="Daily_return",histnorm='percent')

In [14]:


df['Volumn'] = df['<Volume>']/1000000
Volumnshort_rolling = df['Volumn'].rolling(window=7).mean()

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datetime, y=df['Daily_return'], 
                        mode='lines', name='Dailly_Retun'))
fig.add_trace(go.Scatter(x=df.datetime, y=df['Volumn'], 
                        mode='lines+markers', name='Volumn'))

fig.add_trace(go.Scatter(x=df.datetime, y=Volumnshort_rolling, 
                        mode='lines', name='Rolling_volumn'))

In [15]:
df['Volumn'].head(15).describe()

count    15.000000
mean     10.358473
std       2.284296
min       7.413700
25%       8.320950
50%      10.420800
75%      11.045700
max      14.957000
Name: Volumn, dtype: float64

In [16]:
df['Volumn'].head(10).describe()

count    10.000000
mean     10.019830
std       2.222011
min       7.518800
25%       8.291125
50%      10.246250
75%      10.961275
max      14.957000
Name: Volumn, dtype: float64